In [ ]:
import pandas as pd
import re
import nltk
import string
import tensorflow as tf
import keras
import warnings
import gensim.downloader as api
import io
warnings.filterwarnings('ignore')
# from keras_preprocessing.sequence import pad_sequences
# from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
# from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import WordNetLemmatizer
from bs4 import BeautifulSoup
# from parallel_pandas import ParallelPandas
#initialize parallel-pandas
# ParallelPandas.initialize(n_cpu=16, split_factor=4, disable_pr_bar=False)
# from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import f1_score,classification_report,accuracy_score
from sklearn.metrics import precision_score,recall_score
from sklearn.metrics import roc_auc_score
from sklearn import metrics
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import Dense, Input, Dropout, Conv1D, MaxPooling1D, LSTM, Bidirectional, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.keras.models import Model
# from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import BatchNormalization,Activation
from tensorflow.keras.layers import GRU

In [ ]:
# tf.config.set_soft_device_placement(False)
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
from tensorflow.keras.layers import  GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate

In [ ]:
from tensorflow.keras.layers import  MaxPool1D, Flatten

In [ ]:
from tensorflow.keras.constraints import MaxNorm

In [ ]:
from sklearn.utils import class_weight

In [ ]:
import tensorflow.keras.utils

In [ ]:
# !pip install -U numpy==1.11.0.

In [ ]:
class MyValidationError(Exception):
    pass


In [ ]:
nltk.download('all')

In [ ]:
# from tensorflow.python.compiler.mlcompute import mlcompute

# tf.compat.v1.disable_eager_execution()
# mlcompute.set_mlc_device(device_name='gpu')
# from tensorflow.mlcompute import mlcompute
# !python -m numpy

In [ ]:
print(tf.config.list_physical_devices())

In [ ]:
# print(np.__version__)
# !pip install mlcompute
# !pip install tensorflow-gpu
# !pip install parallel-pandas
# !pip install tensorflow
# !pip install nltk
# !pip install gensim
# !python -m pip install sklearn
# !pip install -U scikit-learn scipy matplotlib

In [ ]:
lemmatizer = WordNetLemmatizer()
stop_words = stopwords.words('english')

In [ ]:
# Functions for data pre-processing
def toLowerCase(text):
#     print("Started function toLowerCase")
    return str.lower(text)

def remove_html(text):
#     print("Started function remove_html")
    new_soup = BeautifulSoup(text,'html.parser')
    text = new_soup.get_text()
    return text

def remove_punctuation(text):
#     print("Started function remove_punctuation")
    text = " ".join([word for word in text if word not in string.punctuation])
    return text

def url_replace(text):
#     print("Started function url_replace")
#     re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", text)
    matches = re.findall(r'http\S+', text)
#     print(matches)

    for match in matches:
        text = text.replace(match, '')
    
    matches = re.findall(r'www.\S+', text)    
    
    for match in matches:
        text = text.replace(match, '')
    
    return text

def whiteSpaceRemoval(text):
#     print("Started function whiteSpaceRemoval")
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    return text

def specialCharacterRemoval(text):
#     print("Started function specialCharacterRemoval")
    text = re.sub(r'\W', ' ', str(text))
    return text

def singleCharRemoval(text):
#     print("Started function singleCharRemoval")
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    return text

def numbers_cleanup(x):
#     print("Started function numbers_cleanup")
    if bool(re.search(r'\d', x)):
        x = re.sub('[0-9]{5,}', '#####', x)
        x = re.sub('[0-9]{4}', '####', x)
        x = re.sub('[0-9]{3}', '###', x)
        x = re.sub('[0-9]{2}', '##', x)
    return x

def lemmatizeStopWordAndSmallWordRemoval(text):
#     print("Started function lemmatizeStopWordAndSmallWordRemoval")
    words = text.split()
    words_lemmatized = [lemmatizer.lemmatize(word=word) for word in words]
    words_lemmatized_nostop = [word for word in words_lemmatized if word not in stop_words]
    # for word in words_lemmatized:
    #     if word not in stopwords.words():
    #         words_lemmatized_nostop.append(word)
    # words_lemmatized_nostop = [word for word.words() in words_lemmatized if word not in stopwords]
    word_lemmatized_nostop_lessthan3 = [word for word in words_lemmatized_nostop if len(word)>3]
    text = ' '.join(word_lemmatized_nostop_lessthan3)
    return text
    
    

In [ ]:
# Define a function to compute the max length of sequence
def max_length(sequences):
    '''
    input:
        sequences: a 2D list of integer sequences
    output:
        max_length: the max length of the sequences
    '''
    max_length = 0
    for i, seq in enumerate(sequences):
        length = len(seq)
        if max_length < length:
            max_length = length
    return max_length

In [ ]:
def preProcessDataMBTI(df_mbti,approach):
    print("Started execution of function preProcessDataMBTI")
    print("Number of columns:",df_mbti.columns)
    print("Number of records:",len(df_mbti))
    print("Number of records with personality type assigned: ",len(df_mbti['type'].notnull()))
    df_mbti['len'] = df_mbti['posts'].apply(lambda s : len(s))
#     df_mbti['len'].plot.hist(bins=100)
    maximum_len1 = int(df_mbti.len.quantile(0.9))
    lemmatizer = WordNetLemmatizer()
    stop_words = stopwords.words('english')
    print("Class-wise break-up of personality types:\n",df_mbti.type.value_counts())
    with tf.device("/TPU:0"):
        df_mbti['posts'] = df_mbti["posts"].apply(lambda x: toLowerCase(x))
        df_mbti['posts'] = df_mbti['posts'].apply(lambda x: remove_html(x))
        df_mbti['posts'] = df_mbti['posts'].apply(lambda x: url_replace(x))
        df_mbti['posts'] = df_mbti['posts'].apply(lambda x: word_tokenize(x))
        df_mbti['posts'] = df_mbti["posts"].apply(lambda x: specialCharacterRemoval(x))
        # df_mbti['posts'] = df_mbti["posts"].apply(lambda x: remove_punctuation(x))
        df_mbti['posts'] = df_mbti["posts"].apply(lambda x: numbers_cleanup(x))
        df_mbti['posts'] = df_mbti["posts"].apply(lambda x: whiteSpaceRemoval(x))
        df_mbti['posts'] = df_mbti["posts"].apply(lambda x: singleCharRemoval(x))
        df_mbti['posts'] = df_mbti["posts"].apply(lambda x: lemmatizeStopWordAndSmallWordRemoval(x))
        
#         size_vocabulary = 10000
        token_oov = '<OOV>'
        trunc_type='post'
        padding_type='post'
        embedding_dimension = 300
        x_train, x_test, y_train, y_test = train_test_split(df_mbti['posts'], df_mbti['type'],
                                                        stratify=df_mbti['type'], 
                                                        test_size=0.30)

        x_test, x_holdout, y_test, y_holdout = train_test_split(x_test, y_test,
                                                        stratify=y_test, 
                                                        test_size=0.33)
    #     text_tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = size_vocabulary, oov_token=token_oov)
        class_weights_train = class_weight.compute_class_weight(class_weight ='balanced',classes=np.unique(y_train),y=y_train)
        class_weights_test = class_weight.compute_class_weight(class_weight ='balanced',classes=np.unique(y_test),y=y_test)
        class_weights_train = dict(enumerate(class_weights_train))
        class_weights_test = dict(enumerate(class_weights_test))
    #     class_weights = class_weight.compute_class_weight(class_weight ='balanced',classes=np.unique(y_train),y=y_train)
    #     class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}
    #     class_weights = dict(enumerate(class_weights))
    
        text_tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token=token_oov)#,num_words=size_vocabulary)
        text_tokenizer.fit_on_texts(x_train)
        
        print("Sample tokens \n",dict(list(text_tokenizer.word_index.items())[:15]))
        
        x_train_sequences = text_tokenizer.texts_to_sequences(x_train)
        if approach is 'new':
            maximum_len2 = max_length(x_train_sequences)
            maximum_len = min(maximum_len1,maximum_len2)
        else:
            maximum_len = maximum_len1
    #     print("Tokenized training data sample: \n",x_train_sequences[5])
        x_train_padded = tf.keras.preprocessing.sequence.pad_sequences(x_train_sequences,maxlen=maximum_len,
                                                                            truncating=trunc_type,
                                                                            padding=padding_type)
        print("Sample training data padded length is: ",len(x_train_padded[5]))
        x_test_sequences = text_tokenizer.texts_to_sequences(x_test)
        x_test_padded = tf.keras.preprocessing.sequence.pad_sequences(x_test_sequences,maxlen=maximum_len,
                                                                            truncating=trunc_type,
                                                                            padding=padding_type)
        x_holdout_sequences = text_tokenizer.texts_to_sequences(x_holdout)
        x_holdout_padded = tf.keras.preprocessing.sequence.pad_sequences(x_holdout_sequences,maxlen=maximum_len,
                                                                            truncating=trunc_type,
                                                                            padding=padding_type)
        label_encoder = LabelEncoder()
        label_encoder.fit(y_train)
        y_train_encoded = label_encoder.transform(y_train.values)
        y_test_encoded = label_encoder.transform(y_test.values)
        y_holdout_encoded = label_encoder.transform(y_holdout.values)

    #     if approach is 'new':
    #         y_train_encoded = tensorflow.keras.utils.to_categorical(y_train_encoded)
    #         y_test_encoded = tensorflow.keras.utils.to_categorical(y_test_encoded)
    #         y_holdout_encoded = tensorflow.keras.utils.to_categorical(y_holdout_encoded)

        print("Shape of training data with padding: ", x_train_padded.shape)
        print("Shape of testing data with padding: ", x_test_padded.shape)
        print("Shape of holdout data with padding: ", x_holdout_padded.shape)
        print("Shape of training labels encoded: ", y_train_encoded.shape)
        print("Shape of testing labels encoded: ", y_test_encoded.shape)
        print("Shape of holdout labels encoded: ", y_holdout_encoded.shape)
        num_labels = len(df_mbti['type'].unique())
        print("Number of labels: ",num_labels)

        return x_train_padded, x_test_padded, x_holdout_padded, y_train_encoded, y_test_encoded, y_holdout_encoded, text_tokenizer, maximum_len, embedding_dimension, label_encoder, num_labels, class_weights_train, class_weights_test

In [ ]:
def removeColons(text):
    text = text.replace(':','')
    return str(text)

In [ ]:
def preProcessDataTEC(df_tec,approach):
    print("Started execution of function preProcessDataTEC")
#     df_tec = df_tec.rename(columns={0:"tweets",1:"emotions"},errors="raise")
    df_tec["tweets"] = df_tec["tweets"].astype('str')
    df_tec["emotions"] = df_tec["emotions"].astype('str')    
    df_tec['emotions'] = df_tec['emotions'].apply(lambda x: removeColons(x))    
    print("Number of columns:",df_tec.columns)
    print("Number of records:",len(df_tec))
    print("Number of records with emotion assigned: ",len(df_tec['emotions'].notnull()))
    print(df_tec.dtypes)
    df_tec['len'] = df_tec['tweets'].apply(lambda s : len(s))
#     df_tec['len'].plot.hist(bins=100)
    maximum_len1 = int(df_tec.len.quantile(0.9))
    
    lemmatizer = WordNetLemmatizer()
    stop_words = stopwords.words('english')
    print("Class-wise break-up of emotions:\n",df_tec.emotions.value_counts())
    with tf.device("/TPU:0"):
        df_tec['tweets'] = df_tec["tweets"].apply(lambda x: toLowerCase(x))
        df_tec['tweets'] = df_tec['tweets'].apply(lambda x: remove_html(x))
        df_tec['tweets'] = df_tec['tweets'].apply(lambda x: url_replace(x))
        df_tec['tweets'] = df_tec['tweets'].apply(lambda x: word_tokenize(x))
        df_tec['tweets'] = df_tec["tweets"].apply(lambda x: specialCharacterRemoval(x))
        # df_tec['tweets'] = df_tec["tweets"].apply(lambda x: remove_punctuation(x))
        df_tec['tweets'] = df_tec["tweets"].apply(lambda x: numbers_cleanup(x))
        df_tec['tweets'] = df_tec["tweets"].apply(lambda x: whiteSpaceRemoval(x))
        df_tec['tweets'] = df_tec["tweets"].apply(lambda x: singleCharRemoval(x))
        df_tec['tweets'] = df_tec["tweets"].apply(lambda x: lemmatizeStopWordAndSmallWordRemoval(x))
        
        
        token_oov = '<OOV>'
        trunc_type='post'
        padding_type='post'
        embedding_dimension = 300
        x_train, x_test, y_train, y_test = train_test_split(df_tec['tweets'], df_tec['emotions'],
                                                        stratify=df_tec['emotions'], 
                                                        test_size=0.30)

        x_test, x_holdout, y_test, y_holdout = train_test_split(x_test, y_test,
                                                        stratify=y_test, 
                                                        test_size=0.33)
    #     text_tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = size_vocabulary, oov_token=token_oov)
    
        class_weights_train = class_weight.compute_class_weight(class_weight ='balanced',classes=np.unique(y_train),y=y_train)
        class_weights_test = class_weight.compute_class_weight(class_weight ='balanced',classes=np.unique(y_test),y=y_test)
        class_weights_train = dict(enumerate(class_weights_train))
        class_weights_test = dict(enumerate(class_weights_test))
        
        text_tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token=token_oov)
        text_tokenizer.fit_on_texts(x_train)
        
        print("Sample tokens \n",dict(list(text_tokenizer.word_index.items())[:15]))
        x_train_sequences = text_tokenizer.texts_to_sequences(x_train)
        if approach is 'new':
            maximum_len2 = max_length(x_train_sequences)
            maximum_len = min(maximum_len1,maximum_len2)
        else:
            maximum_len = maximum_len1
    
        x_train_padded = tf.keras.preprocessing.sequence.pad_sequences(x_train_sequences,maxlen=maximum_len,
                                                                           truncating=trunc_type,
                                                                           padding=padding_type)
        print("Sample training data padded length: ",len(x_train_padded[5]))
        x_test_sequences = text_tokenizer.texts_to_sequences(x_test)
        x_test_padded = tf.keras.preprocessing.sequence.pad_sequences(x_test_sequences,maxlen=maximum_len,
                                                                            truncating=trunc_type,
                                                                            padding=padding_type)
        x_holdout_sequences = text_tokenizer.texts_to_sequences(x_holdout)
        x_holdout_padded = tf.keras.preprocessing.sequence.pad_sequences(x_holdout_sequences,maxlen=maximum_len,
                                                                            truncating=trunc_type,
                                                                            padding=padding_type)
        label_encoder = LabelEncoder()
        label_encoder.fit(y_train)
        y_train_encoded = label_encoder.transform(y_train.values)
        y_test_encoded = label_encoder.transform(y_test.values)
        y_holdout_encoded = label_encoder.transform(y_holdout.values)
        
#         y_train_encoded = tensorflow.keras.utils.to_categorical(y_train_encoded)
#         y_test_encoded = tensorflow.keras.utils.to_categorical(y_test_encoded)
#         y_holdout_encoded = tensorflow.keras.utils.to_categorical(y_holdout_encoded)

        print("Shape of training data with padding: ", x_train_padded.shape)
        print("Shape of testing data with padding: ", x_test_padded.shape)
        print("Shape of holdout data with padding: ", x_holdout_padded.shape)
        print("Shape of training labels encoded: ", y_train_encoded.shape)
        print("Shape of testing labels encoded: ", y_test_encoded.shape)
        print("Shape of holdout labels encoded: ", y_holdout_encoded.shape)
        num_labels = len(df_tec['emotions'].unique())
        print("Number of labels: ",num_labels)

        return x_train_padded, x_test_padded, x_holdout_padded, y_train_encoded, y_test_encoded, y_holdout_encoded, text_tokenizer, maximum_len, embedding_dimension, label_encoder, num_labels, class_weights_train, class_weights_test

In [ ]:
def preProcessDataISEAR(df_isear,approach):
    print("Started execution of function preProcessDataISEAR")
    df_isear = df_isear.rename(columns={"Field1":"emotions","SIT":"answers"},errors="raise")
#     df_isear['emotions'] = df_isear['emotions'].apply(lambda x: removeColons(x))    
    df_isear['emotions'] = df_isear['emotions'].astype('str')
    df_isear['answers'] = df_isear['answers'].astype('str')
    print("Number of columns:",df_isear.columns)
    print("Number of records:",len(df_isear))
    print("Number of records with emotion assigned: ",len(df_isear['emotions'].notnull()))
    df_isear['len'] = df_isear['answers'].apply(lambda s : len(s))
#     df_isear['len'].plot.hist(bins=100)
    maximum_len1 = int(df_isear.len.quantile(0.9))
    lemmatizer = WordNetLemmatizer()
    stop_words = stopwords.words('english')
    
    print("Class-wise break-up of emotions:\n",df_isear.emotions.value_counts())
    with tf.device("/TPU:0"):
        df_isear['answers'] = df_isear["answers"].apply(lambda x: toLowerCase(x))
        df_isear['answers'] = df_isear['answers'].apply(lambda x: remove_html(x))
        df_isear['answers'] = df_isear['answers'].apply(lambda x: url_replace(x))
        df_isear['answers'] = df_isear['answers'].apply(lambda x: word_tokenize(x))
        df_isear['answers'] = df_isear["answers"].apply(lambda x: specialCharacterRemoval(x))
        # df_isear['answers'] = df_isear["answers"].apply(lambda x: remove_punctuation(x))
        df_isear['answers'] = df_isear["answers"].apply(lambda x: numbers_cleanup(x))
        df_isear['answers'] = df_isear["answers"].apply(lambda x: whiteSpaceRemoval(x))
        df_isear['answers'] = df_isear["answers"].apply(lambda x: singleCharRemoval(x))
        df_isear['answers'] = df_isear["answers"].apply(lambda x: lemmatizeStopWordAndSmallWordRemoval(x))
    
        token_oov = '<OOV>'
        trunc_type='post'
        padding_type='post'
        embedding_dimension = 300

        x_train, x_test, y_train, y_test = train_test_split(df_isear['answers'], df_isear['emotions'],
                                                        stratify=df_isear['emotions'], 
                                                        test_size=0.30)

        x_test, x_holdout, y_test, y_holdout = train_test_split(x_test, y_test,
                                                        stratify=y_test, 
                                                        test_size=0.33)

        class_weights_train = class_weight.compute_class_weight(class_weight ='balanced',classes=np.unique(y_train),y=y_train)
        class_weights_test = class_weight.compute_class_weight(class_weight ='balanced',classes=np.unique(y_test),y=y_test)
        class_weights_train = dict(enumerate(class_weights_train))
        class_weights_test = dict(enumerate(class_weights_test))

        text_tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token=token_oov)
        text_tokenizer.fit_on_texts(x_train)
        print("Sample tokens \n",dict(list(text_tokenizer.word_index.items())[:15]))
        x_train_sequences = text_tokenizer.texts_to_sequences(x_train)

        if approach is 'new':
            maximum_len2 = max_length(x_train_sequences)
            maximum_len = min(maximum_len1,maximum_len2)
        else:
            maximum_len = maximum_len1

        x_train_padded = tf.keras.preprocessing.sequence.pad_sequences(x_train_sequences,maxlen=maximum_len,
                                                                           truncating=trunc_type,
                                                                           padding=padding_type)

        print("Sample training data padded length: ",len(x_train_padded[5]))

        x_test_sequences = text_tokenizer.texts_to_sequences(x_test)
        x_test_padded = tf.keras.preprocessing.sequence.pad_sequences(x_test_sequences,maxlen=maximum_len, padding=padding_type,truncating=trunc_type)

        x_holdout_sequences = text_tokenizer.texts_to_sequences(x_holdout)
        x_holdout_padded = tf.keras.preprocessing.sequence.pad_sequences(x_holdout_sequences,maxlen=maximum_len, padding=padding_type,truncating=trunc_type)

        label_encoder = LabelEncoder()
        label_encoder.fit(y_train.values)
        y_train_encoded = label_encoder.transform(y_train.values)
        y_test_encoded = label_encoder.transform(y_test.values)
        y_holdout_encoded = label_encoder.transform(y_holdout.values)

    #     y_train_encoded = tensorflow.keras.utils.to_categorical(y_train_encoded)
    #     y_test_encoded = tensorflow.keras.utils.to_categorical(y_test_encoded)
    #     y_holdout_encoded = tensorflow.keras.utils.to_categorical(y_holdout_encoded)

        print("Shape of training data with padding: ", x_train_padded.shape)
        print("Shape of testing data with padding: ", x_test_padded.shape)
        print("Shape of holdout data with padding: ", x_holdout_padded.shape)
        print("Shape of training labels encoded: ", y_train_encoded.shape)
        print("Shape of testing labels encoded: ", y_test_encoded.shape)
        print("Shape of holdout labels encoded: ", y_holdout_encoded.shape)
        num_labels = len(df_isear['emotions'].unique())
        print("Number of labels: ",num_labels)

        return x_train_padded, x_test_padded, x_holdout_padded, y_train_encoded, y_test_encoded, y_holdout_encoded, text_tokenizer, maximum_len, embedding_dimension, label_encoder, num_labels, class_weights_train, class_weights_test
    

In [ ]:
def readDataAndPreprocess(datasetName,approach):
    print("Started function exeuction of readDataAndPreprocess")
    if datasetName is 'MBTI':
        df_data = pd.read_csv('/kaggle/input/mbtinew/mbti_1.csv')#('gdrive/MyDrive/mbti_1.csv')
        return preProcessDataMBTI(df_data,approach)
        
    elif datasetName is 'TEC':
        # 
        df_data = pd.read_csv('/kaggle/input/kaggleinputtwitteremotioncorpus-1/tec.csv')
        return preProcessDataTEC(df_data,approach)
    elif datasetName is 'ISEAR':
        # 
        df_data = pd.read_excel('/kaggle/input/isearfortextclassification/ISEAR.xlsx')        
        return preProcessDataISEAR(df_data,approach)
    else:
        raise MyValidationError("Invalid input dataset name.")        
    
    # return df_data_preprocessed

In [ ]:
def create_embeddings_glove(dir_glove): 
    print("Started execution of function create_embeddings_glove")
    embeddings_index = {}
    f = open(dir_glove,encoding="utf8")
    for line in f:
        values = line.split()
        word = values[0]
        embeddings_index[word] = np.asarray(values[1:],dtype='float32')
    f.close()

    return embeddings_index

In [ ]:


def loadFastTextEmbeddings(fname):
    print("Started execution of function loadFastTextEmbeddings")
#     fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
#     n, d = map(int, fin.readline().split())
#     data = {}
#     for line in fin:
#         tokens = line.rstrip().split(' ')
#         data[tokens[0]] = map(float, tokens[1:])
#     return data
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_fasttext = dict(get_coefs(*o.split(" ")) for o in open(fname) if len(o)>100)
    return embeddings_fasttext

In [ ]:
import gensim
from gensim.models import Word2Vec, KeyedVectors

In [ ]:

def loadWord2vecEmbeddings(filePath):
    print("Started execution of function loadWord2vecEmbeddings")
#     embeddings_index = {}
#     f = open(filePath,encoding="utf8")
#     for line in f:
#         values = line.split()
#         word = values[0]
#         embeddings_index[word] = np.asarray(values[1:],dtype='float32')
#     f.close()
    
#     embeddings_index = KeyedVectors.load_word2vec_format(filePath, binary=True, limit=100000)
    embeddings_index = KeyedVectors.load_word2vec_format(filePath, binary=True)
    return embeddings_index
    

In [ ]:
with tf.device("/TPU:0"):
    embeddings_word2vec = loadWord2vecEmbeddings('/kaggle/input/googlenewsword2vec10b300d/GoogleNews-vectors-negative300.bin')
    embeddings_fasttext = loadFastTextEmbeddings('/kaggle/input/fasttextwikinews300d1msubword/wiki-news-300d-1M-subword.vec')
    indexed_embeddings_glove = create_embeddings_glove(f"/kaggle/input/glove42b300/glove.42B.300d.txt")


In [ ]:
def getEmbeddingMatrixForGlove(text_tokenizer, embedding_dimension,indexed_embeddings_glove):
    print("Started function execution of getEmbeddingMatrixForGlove")
#     dir_glove = f"/kaggle/input/glove42b300/glove.42B.300d.txt"#f"gdrive/MyDrive/glove.42B.{embedding_dimension}d.txt" #f"./glove.42B.{embedding_dimension}d.txt"
#     indexed_embeddings_glove = create_embeddings_glove(dir_glove)
    words_index = text_tokenizer.word_index
    # embedding_dimension = 300
    # words_index
    matrix_embeddings = np.zeros((len(words_index)+1,embedding_dimension))
    for word,idx in words_index.items():
        vector_embeddings = indexed_embeddings_glove.get(word)
        if vector_embeddings is not None:
            matrix_embeddings[idx] = vector_embeddings
            
    print("Shape of the word-embeddings matrix: ",matrix_embeddings.shape)
    return matrix_embeddings
    # 

In [ ]:
# embeddings_fasttext = api.load("fasttext-wiki-news-subwords-300")
# embeddings_fasttext = api.load("fasttext-wiki-news-subwords-300")
# /kaggle/input/fasttextwikinews300d1msubword/wiki-news-300d-1M-subword.vec

In [ ]:
def getEmbeddingMatrixForFasttext(text_tokenizer, embedding_dimension,embeddings_fasttext):
    print("Started execution of function getEmbeddingMatrixForFasttext")
#     embeddings_fasttext = loadFastTextEmbeddings('/kaggle/input/fasttextwikinews300d1msubword/wiki-news-300d-1M-subword.vec')

    words_index = text_tokenizer.word_index
    # embedding_dimension = 300
    # words_index
    print("Started creating the embedding matrix.")
    matrix_embeddings = np.zeros((len(words_index)+1,embedding_dimension))
    for word,idx in words_index.items():
#         if word is '<OOV>':
#             word = 'OOV'
#             continue
        vector_embeddings = embeddings_fasttext.get(word)#[word]
        if vector_embeddings is not None:
            matrix_embeddings[idx] = vector_embeddings
            
    print("Shape of the word-embeddings matrix: ",matrix_embeddings.shape)
    return matrix_embeddings
    
    

In [ ]:
def getEmbeddingMatrixForWord2Vec(text_tokenizer, embedding_dimension,embeddings_word2vec):
    print("Started execution of function getEmbeddingMatrixForWord2Vec")
#     embeddings_word2vec = loadWord2vecEmbeddings('/kaggle/input/googlenewsword2vec10b300d/GoogleNews-vectors-negative300.bin')#('/kaggle/input/fasttextwikinews300d1msubword/wiki-news-300d-1M-subword.vec')

    words_index = text_tokenizer.word_index
    # embedding_dimension = 300
    # words_index
    print("Started creating the embedding matrix.")
    matrix_embeddings = np.zeros((len(words_index)+1,embedding_dimension))
    for word,idx in words_index.items():
#         if word is '<OOV>':
#             word = 'OOV'
#             continue
        try:
            vector_embeddings = embeddings_word2vec[word]#.get(word)#[word]
        except KeyError:
            continue
        if vector_embeddings is not None:
            matrix_embeddings[idx] = vector_embeddings
            
    print("Shape of the word-embeddings matrix: ",matrix_embeddings.shape)
    return matrix_embeddings

In [ ]:
def createModelCNNBILSTM(num_filters, kernel_size, text_tokenizer, embedding_dimension, maximum_len,matrix_embeddings,num_labels,optimizer):
    with tpu_strategy.scope():
        print("Started execution of function createModelCNNBILSTM")
        print("Number of filters: ",num_filters)
        print("Number of kernels: ",kernel_size)
        print("Optimizer is: ",optimizer)
    #     if optimizer is 'Adam':
    #         opt = 'adam'        
    #     elif optimizer is 'SGD':
    #         opt = 'sgd'
    #     elif optimizer is 'adadelta':
    #         opt = 'adadelta'
    #     else:
    #         raise MyValidationError("Invalid optimizer name.")

        # input_sequence = Input(shape=(9050,), dtype='int32')    
        # layer_embedding = Embedding(len(words_index) + 1,embedding_dimension,weights=[matrix_embeddings],input_length=maximum_len,trainable=False)

        input_sequence = Input(shape=(maximum_len,), dtype='int32')
        layer_embedding = Embedding(len(text_tokenizer.word_index) + 1,embedding_dimension,weights=[matrix_embeddings],input_length=maximum_len,trainable=False)
        sequences_embedded = layer_embedding(input_sequence)
        layer_conv1 = Conv1D(num_filters, kernel_size, activation='relu')(sequences_embedded)
    #     26th Apr
        layer_pool1 = MaxPooling1D(pool_size=5,padding='same')(layer_conv1)
    #     layer_pool1 = MaxPooling1D(pool_size=2,strides=2)(layer_conv1)
    #  26th Apr
        layer_drop1 = Dropout(0.5)(layer_pool1)
        layer_drop1 = Dropout(0.5)(layer_drop1)

    #     layer_drop1 = Dropout(0.5)(layer_drop1)
        layer_bn1 = BatchNormalization()(layer_drop1)

        layer_conv2 = Conv1D(num_filters, kernel_size, activation='relu')(layer_bn1)

    #     layer_pool2 = MaxPooling1D(pool_size=5)(layer_conv2)

        layer_pool2 = MaxPooling1D(pool_size=5,padding='same')(layer_conv2)
        layer_drop2 = Dropout(0.5)(layer_pool2)
        layer_drop2 = Dropout(0.5)(layer_drop2)

    #     layer_drop2 = Dropout(0.5)(layer_drop2)

        layer_bn2 = BatchNormalization()(layer_drop2)

        # layer_conv3 = Conv1D(128, 5, activation='relu')(layer_bn2)
        # layer_pool3 = MaxPooling1D(pool_size=5)(layer_conv3)
        # layer_drop3 = Dropout(0.5)(layer_pool3)
        # layer_bn3 = BatchNormalization()(layer_drop3)

        layer_bilstm = Bidirectional(LSTM(num_filters,return_sequences=False,dropout=0.2,recurrent_dropout=0.2))(layer_bn2)#(layer_drop2)#(layer_pool3)

        # lstm_layer1 = Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))(layer_bn3)#(pooling_layer2)
        # lstm_layer2 = Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2))(lstm_layer1)
    #     layer_drop3 = Dropout(0.5)(layer_bilstm)
    #     layer_drop3 = Dropout(0.5)(layer_drop3)
    #     27th Apr
    #     layer_dense1 = Dense(512,activation='sigmoid')(layer_drop3)
    #     27th Apr

        layer_drop3 = Dropout(0.5)(layer_bilstm)
        layer_drop3 = Dropout(0.5)(layer_drop3)

    #     layer_dense1 = Dense(256,activation='relu')(layer_bilstm)
    #     layer_drop4 = Dropout(0.5)(layer_dense1)

        layer_output = Dense(num_labels,activation='softmax')(layer_drop3)#(layer_dense1)#(layer_drop3)

        # layer_activation = Activation('softmax')(layer_output)

        model_bilstm = Model(inputs=input_sequence, outputs=layer_output)
        model_bilstm.compile(loss = 'sparse_categorical_crossentropy',metrics=['accuracy'], optimizer=optimizer)
    return model_bilstm
    

In [ ]:
def createModelCNNBIGRU(num_filters, kernel_size, text_tokenizer, embedding_dimension, maximum_len,matrix_embeddings,num_labels,optimizer):
    with tpu_strategy.scope():
        print("Started execution of function createModelCNNBIGRU")
        print("Number of filters: ",num_filters)
        print("Number of kernels: ",kernel_size)
        print("Optimizer is: ",optimizer)
    #     if optimizer is 'Adam':
    #         opt = 'adam'        
    #     elif optimizer is 'SGD':
    #         opt = 'sgd'
    #     elif optimizer is 'adadelta':
    #         opt = 'adadelta'
    #     else:
    #         raise MyValidationError("Invalid optimizer name.")

        # input_sequence = Input(shape=(9050,), dtype='int32')    
        # layer_embedding = Embedding(len(words_index) + 1,embedding_dimension,weights=[matrix_embeddings],input_length=maximum_len,trainable=False)

        input_sequence = Input(shape=(maximum_len,), dtype='int32')
        layer_embedding = Embedding(len(text_tokenizer.word_index) + 1,embedding_dimension,weights=[matrix_embeddings],input_length=maximum_len,trainable=False)
        sequences_embedded = layer_embedding(input_sequence)
        layer_conv1 = Conv1D(num_filters, kernel_size, activation='relu')(sequences_embedded)
    #     26th Apr
        layer_pool1 = MaxPooling1D(pool_size=5,padding='same')(layer_conv1)
    #     layer_pool1 = MaxPooling1D(pool_size=2,strides=2)(layer_conv1)
    #  26th Apr
        layer_drop1 = Dropout(0.5)(layer_pool1)
        layer_drop1 = Dropout(0.5)(layer_drop1)

    #     layer_drop1 = Dropout(0.5)(layer_drop1)
        layer_bn1 = BatchNormalization()(layer_drop1)

        layer_conv2 = Conv1D(num_filters, kernel_size, activation='relu')(layer_bn1)

    #     layer_pool2 = MaxPooling1D(pool_size=5)(layer_conv2)

        layer_pool2 = MaxPooling1D(pool_size=5,padding='same')(layer_conv2)
        layer_drop2 = Dropout(0.5)(layer_pool2)
        layer_drop2 = Dropout(0.5)(layer_drop2)

    #     layer_drop2 = Dropout(0.5)(layer_drop2)

        layer_bn2 = BatchNormalization()(layer_drop2)

        # layer_conv3 = Conv1D(128, 5, activation='relu')(layer_bn2)
        # layer_pool3 = MaxPooling1D(pool_size=5)(layer_conv3)
        # layer_drop3 = Dropout(0.5)(layer_pool3)
        # layer_bn3 = BatchNormalization()(layer_drop3)

        layer_bilstm = Bidirectional(GRU(num_filters,return_sequences=False,dropout=0.2,recurrent_dropout=0.2))(layer_bn2)#(layer_drop2)#(layer_pool3)

        # lstm_layer1 = Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))(layer_bn3)#(pooling_layer2)
        # lstm_layer2 = Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2))(lstm_layer1)
    #     layer_drop3 = Dropout(0.5)(layer_bilstm)
    #     layer_drop3 = Dropout(0.5)(layer_drop3)
    #     27th Apr
    #     layer_dense1 = Dense(512,activation='sigmoid')(layer_drop3)
    #     27th Apr

        layer_drop3 = Dropout(0.5)(layer_bilstm)
        layer_drop3 = Dropout(0.5)(drop3)

    #     layer_dense1 = Dense(256,activation='relu')(layer_bilstm)
    #     layer_drop4 = Dropout(0.5)(layer_dense1)

        layer_output = Dense(num_labels,activation='softmax')(layer_drop3)#(layer_dense1)#(layer_drop3)

        # layer_activation = Activation('softmax')(layer_output)

        model_bilstm = Model(inputs=input_sequence, outputs=layer_output)
        model_bilstm.compile(loss = 'sparse_categorical_crossentropy',metrics=['accuracy'], optimizer=optimizer)
    return model_bilstm


In [ ]:
def createModelCNNBILSTM3(num_filters, kernel_size, text_tokenizer, embedding_dimension, maximum_len,matrix_embeddings,num_labels,optimizer):
    with tpu_strategy.scope():
        print("Started execution of function createModelCNNBILSTM3")
        print("Number of filters: ",num_filters)
        print("Number of kernels: ",kernel_size)
        print("Optimizer is: ",optimizer)
    #     if optimizer is 'Adam':
    #         opt='adam'#opt = Adam(learning_rate=0.1)
    #     elif optimizer is 'SGD':
    #         opt = SGD(learning_rate=0.001)#, momentum=0.9)#SGD(learning_rate=0.01, momentum=0.9)
    #     elif optimizer is 'adadelta':
    #         opt = 'adadelta'
    #     else:
    #         raise MyValidationError("Invalid optimizer name.")

        # input_sequence = Input(shape=(9050,), dtype='int32')
        input_sequence = Input(shape=(maximum_len,), dtype='int32')
        # layer_embedding = Embedding(len(words_index) + 1,embedding_dimension,weights=[matrix_embeddings],input_length=maximum_len,trainable=False)
        layer_embedding = Embedding(len(text_tokenizer.word_index) + 1,embedding_dimension,weights=[matrix_embeddings],input_length=maximum_len,trainable=False)
        sequences_embedded = layer_embedding(input_sequence)
        layer_conv1 = Conv1D(num_filters, kernel_size, activation='relu')(sequences_embedded)
        #1layer_pool1 = GlobalMaxPooling1D(pool_size=5)(layer_conv1)
        #1layer_drop1 = Dropout(0.5)(layer_pool1)
        #1layer_bn1 = BatchNormalization()(layer_drop1)
        #1layer_conv2 = Conv1D(num_filters, kernel_size, activation='relu')(layer_bn1)
        #1layer_pool2 = MaxPooling1D(pool_size=5)(layer_conv2)
        #1layer_drop2 = Dropout(0.5)(layer_pool2)
        #1layer_bn2 = BatchNormalization()(layer_drop2)
        # layer_conv3 = Conv1D(128, 5, activation='relu')(layer_bn2)
        # layer_pool3 = MaxPooling1D(pool_size=5)(layer_conv3)
        # layer_drop3 = Dropout(0.5)(layer_pool3)
        # layer_bn3 = BatchNormalization()(layer_drop3)
        layer_bilstm = Bidirectional(LSTM(num_filters,return_sequences=True,dropout=0.2,recurrent_dropout=0.2))(layer_conv1)#(layer_pool1)#(layer_bn1)#(layer_bn2)#(layer_drop2)#(layer_pool3)
        avg_pool = GlobalAveragePooling1D()(layer_bilstm)
        max_pool = GlobalMaxPooling1D()(layer_bilstm)
        conc = concatenate([avg_pool, max_pool])
        # lstm_layer1 = Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))(layer_bn3)#(pooling_layer2)
        # lstm_layer2 = Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2))(lstm_layer1)
    #     layer_drop3 = Dropout(0.5)(conc)#(layer_bigru)
        layer_dense1 = Dense(embedding_dimension,activation='relu')(conc)#(layer_drop3)
        layer_drop1 = Dropout(0.5)(layer_dense1)
        layer_drop1 = Dropout(0.5)(layer_drop1)
        layer_output = Dense(num_labels,activation='softmax')(layer_drop1)#(layer_dense1)#(layer_drop3)
        # layer_activation = Activation('softmax')(layer_output)
        model_bilstm = Model(inputs=input_sequence, outputs=layer_output)
        model_bilstm.compile(loss = 'sparse_categorical_crossentropy',metrics=['accuracy'], optimizer=optimizer)
    return model_bilstm
    

In [ ]:
def createModelCNNBIGRU3(num_filters, kernel_size, text_tokenizer, embedding_dimension, maximum_len,matrix_embeddings,num_labels,optimizer):
    with tpu_strategy.scope():
        print("Started execution of function createModelCNNBIGRU3")
        print("Number of filters: ",num_filters)
        print("Number of kernels: ",kernel_size)
        print("Optimizer is: ",optimizer)
    #     if optimizer is 'Adam':
    #         opt='adam'#opt = Adam(learning_rate=0.1)
    #     elif optimizer is 'SGD':
    #         opt = SGD(learning_rate=0.001)#, momentum=0.9)#SGD(learning_rate=0.01, momentum=0.9)
    #     elif optimizer is 'adadelta':
    #         opt = 'adadelta'
    #     else:
    #         raise MyValidationError("Invalid optimizer name.")

        # input_sequence = Input(shape=(9050,), dtype='int32')
        input_sequence = Input(shape=(maximum_len,), dtype='int32')
        # layer_embedding = Embedding(len(words_index) + 1,embedding_dimension,weights=[matrix_embeddings],input_length=maximum_len,trainable=False)
        layer_embedding = Embedding(len(text_tokenizer.word_index) + 1,embedding_dimension,weights=[matrix_embeddings],input_length=maximum_len,trainable=False)
        sequences_embedded = layer_embedding(input_sequence)
        layer_conv1 = Conv1D(num_filters, kernel_size, activation='relu')(sequences_embedded)
        #1layer_pool1 = GlobalMaxPooling1D(pool_size=5)(layer_conv1)
        #1layer_drop1 = Dropout(0.5)(layer_pool1)
        #1layer_bn1 = BatchNormalization()(layer_drop1)
        #1layer_conv2 = Conv1D(num_filters, kernel_size, activation='relu')(layer_bn1)
        #1layer_pool2 = MaxPooling1D(pool_size=5)(layer_conv2)
        #1layer_drop2 = Dropout(0.5)(layer_pool2)
        #1layer_bn2 = BatchNormalization()(layer_drop2)
        # layer_conv3 = Conv1D(128, 5, activation='relu')(layer_bn2)
        # layer_pool3 = MaxPooling1D(pool_size=5)(layer_conv3)
        # layer_drop3 = Dropout(0.5)(layer_pool3)
        # layer_bn3 = BatchNormalization()(layer_drop3)
        layer_bigru = Bidirectional(GRU(num_filters,return_sequences=True,dropout=0.2,recurrent_dropout=0.2))(layer_conv1)#(layer_pool1)#(layer_bn1)#(layer_bn2)#(layer_drop2)#(layer_pool3)
        avg_pool = GlobalAveragePooling1D()(layer_bigru)
        max_pool = GlobalMaxPooling1D()(layer_bigru)
        conc = concatenate([avg_pool, max_pool])
        # lstm_layer1 = Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))(layer_bn3)#(pooling_layer2)
        # lstm_layer2 = Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2))(lstm_layer1)
    #     layer_drop3 = Dropout(0.5)(conc)#(layer_bigru)
        layer_dense1 = Dense(embedding_dimension,activation='relu')(conc)#'relu')(conc)#(layer_drop3)
        layer_drop1 = Dropout(0.5)(layer_dense1)
        layer_drop1 = Dropout(0.5)(layer_drop1)
        layer_output = Dense(num_labels,activation='softmax')(layer_drop1)#(layer_dense1)#'softmax')(layer_dense1)#(layer_drop3)
        # layer_activation = Activation('softmax')(layer_output)
        model_bigru = Model(inputs=input_sequence, outputs=layer_output)
        model_bigru.compile(loss = 'sparse_categorical_crossentropy',metrics=['accuracy'], optimizer=optimizer)
    return model_bigru
    

In [ ]:
# def createModelCNNBIGRU2(num_filters, kernel_size, text_tokenizer, embedding_dimension, maximum_len,matrix_embeddings,num_labels,optimizer):
# def createModelCNNBIGRU2(DLTechniqueName,x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels,batch_size,optimizer,datasetName,embeddingName,class_weights=None):
def createModelCNNBIGRU2(num_filters, kernel_size, text_tokenizer, embedding_dimension, maximum_len,matrix_embeddings,num_labels,optimizer):
    with tpu_strategy.scope():
    #     num_filters = 100
    #     kernel_size = 3
        print("Started execution of function createModelCNNBIGRU2")
        print("Number of filters: ",num_filters)
        print("Number of kernels: ",kernel_size)
        print("Optimizer is: ",optimizer)
    #     if optimizer is 'Adam':
    # #         opt = Adam(learning_rate=0.001)
    #         opt = 'adam'
    #     elif optimizer is 'SGD':
    #         opt = 'sgd'#SGD(learning_rate=0.01)#, momentum=0.9)#SGD(learning_rate=0.01, momentum=0.9)
    # #         opt = tf.keras.optimizers.legacy.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    #     elif optimizer is 'adadelta':
    #         opt = 'adadelta'
    #     elif optimizer is 'rmsprop':
    #         opt = tensorflow.keras.optimizers.RMSprop(learning_rate=0.1)
    #     elif optimizer is 'adagrad':
    #         opt = 'adagrad'
    #     elif optimizer is 'nadam':
    #         opt = tensorflow.keras.optimizers.Nadam(use_ema=True)#'nadam'
    #     else:
    #         raise MyValidationError("Invalid optimizer name.")

            # Channel 1
        input_dim = len(text_tokenizer.word_index)+1#list(matrix_embeddings.shape)[0]#1000
        output_dim = 300#300
        input1 = Input(shape=(maximum_len,))
        embeddding1 = Embedding(
                                input_dim=input_dim, 
                                output_dim=output_dim, 
                                input_length=maximum_len, 
                                input_shape=(maximum_len, ),
                                # Assign the embedding weight with word2vec embedding marix
                                weights = [matrix_embeddings],
                                # Set the weight to be not trainable (static)
                                trainable = False)(input1)
        conv1 = Conv1D(num_filters, kernel_size, activation='relu', 
                       kernel_constraint= MaxNorm( max_value=3, axis=[0,1]))(embeddding1)
        pool1 = MaxPool1D(pool_size=2, strides=2,padding='same')(conv1)
        flat1 = Flatten()(pool1)
        drop1 = Dropout(0.5)(flat1)
        drop1 = Dropout(0.5)(drop1)
        dense1 = Dense(10, activation='relu')(drop1)
        drop1 = Dropout(0.5)(dense1)
        drop1 = Dropout(0.5)(drop1)
        out1 = Dense(units=num_labels, activation='softmax')(drop1)

        # Channel 2
        input2 = Input(shape=(maximum_len,))
        embeddding2 = Embedding(
            input_dim=input_dim, 
                                output_dim=output_dim, 
                                input_length=maximum_len, 
                                input_shape=(maximum_len, ),
                                # Assign the embedding weight with word2vec embedding marix
                                weights = [matrix_embeddings],
                                # Set the weight to be not trainable (static)
                                trainable = False,
                                mask_zero=True)(input2)
        gru2 = Bidirectional(GRU(64,dropout=0.2,recurrent_dropout=0.2))(embeddding2)
        drop2 = Dropout(0.5)(gru2)
        drop2 = Dropout(0.5)(drop2)
        out2 = Dense(units=num_labels, activation='softmax')(drop2)

        # Merge
        merged = concatenate([out1, out2])

        # Interpretation
        outputs = Dense(units=num_labels, activation='softmax')(merged)
        model = Model(inputs=[input1,input2], outputs=outputs)

        model.compile( loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    #     model.fit([x_train,x_train], y_train, class_weight = class_weights,validation_data= ([x_test,x_test],y_test,class_weights),epochs=100,
    #                 callbacks=[tf.keras.callbacks.EarlyStopping(
    # #                                 monitor='accuracy', patience=10),
    #                            monitor='val_accuracy', min_delta=0,#0.01, 
    #                                  patience=20,# verbose=2, 
    #                                  mode='max', restore_best_weights=True),#start_from_epoch=15),
    # #                                     tensorboard_callback,
    #                                     reduce_lr,
    #                         checkpoints
    #                        ],batch_size=batch_size)#50)

    
    return model

In [ ]:
# def createModelCNNBIGRU2(num_filters, kernel_size, text_tokenizer, embedding_dimension, maximum_len,matrix_embeddings,num_labels,optimizer):
# def createModelCNNBIGRU2(DLTechniqueName,x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels,batch_size,optimizer,datasetName,embeddingName,class_weights=None):
def createModelCNNBILSTM2(num_filters, kernel_size, text_tokenizer, embedding_dimension, maximum_len,matrix_embeddings,num_labels,optimizer):
    with tpu_strategy.scope():
    #     num_filters = 100
    #     kernel_size = 3
        print("Started execution of function createModelCNNBILSTM2")
        print("Number of filters: ",num_filters)
        print("Number of kernels: ",kernel_size)
        print("Optimizer is: ",optimizer)
    #     if optimizer is 'Adam':
    # #         opt = Adam(learning_rate=0.001)
    #         opt = 'adam'
    #     elif optimizer is 'SGD':
    #         opt = 'sgd'#SGD(learning_rate=0.01)#, momentum=0.9)#SGD(learning_rate=0.01, momentum=0.9)
    # #         opt = tf.keras.optimizers.legacy.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    #     elif optimizer is 'adadelta':
    #         opt = 'adadelta'
    #     elif optimizer is 'rmsprop':
    #         opt = tensorflow.keras.optimizers.RMSprop(learning_rate=0.1)
    #     elif optimizer is 'adagrad':
    #         opt = 'adagrad'
    #     elif optimizer is 'nadam':
    #         opt = tensorflow.keras.optimizers.Nadam(use_ema=True)#'nadam'
    #     else:
    #         raise MyValidationError("Invalid optimizer name.")
        input_dim = len(text_tokenizer.word_index)+1#list(matrix_embeddings.shape)[0]#1000
        output_dim = 300#300
        input1 = Input(shape=(maximum_len,))
        embeddding1 = Embedding(
                                input_dim=input_dim, 
                                output_dim=output_dim, 
                                input_length=maximum_len, 
                                input_shape=(maximum_len, ),
                                # Assign the embedding weight with word2vec embedding marix
                                weights = [matrix_embeddings],
                                # Set the weight to be not trainable (static)
                                trainable = False)(input1)
        conv1 = Conv1D(num_filters, kernel_size, activation='relu', 
                       kernel_constraint= MaxNorm( max_value=3, axis=[0,1]))(embeddding1)
        pool1 = MaxPool1D(pool_size=2, strides=2,padding='same')(conv1)
        flat1 = Flatten()(pool1)
        drop1 = Dropout(0.5)(flat1)
        drop1 = Dropout(0.5)(drop1)
        dense1 = Dense(10, activation='relu')(drop1)
        drop1 = Dropout(0.5)(dense1)
        drop1 = Dropout(0.5)(drop1)
        out1 = Dense(units=num_labels, activation='softmax')(drop1)

        # Channel 2
        input2 = Input(shape=(maximum_len,))
        embeddding2 = Embedding(
            input_dim=input_dim, 
                                output_dim=output_dim, 
                                input_length=maximum_len, 
                                input_shape=(maximum_len, ),
                                # Assign the embedding weight with word2vec embedding marix
                                weights = [matrix_embeddings],
                                # Set the weight to be not trainable (static)
                                trainable = False,
                                mask_zero=True)(input2)
        gru2 = Bidirectional(LSTM(64,dropout=0.2,recurrent_dropout=0.2))(embeddding2)
        drop2 = Dropout(0.5)(gru2)
        drop2 = Dropout(0.5)(drop2)
        out2 = Dense(units=num_labels, activation='softmax')(drop2)

        # Merge
        merged = concatenate([out1, out2])

        # Interpretation
        outputs = Dense(units=num_labels, activation='softmax')(merged)
        model = Model(inputs=[input1,input2], outputs=outputs)

        model.compile( loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    #     model.fit([x_train,x_train], y_train, class_weight = class_weights,validation_data= ([x_test,x_test],y_test,class_weights),epochs=100,
    #                 callbacks=[tf.keras.callbacks.EarlyStopping(
    # #                                 monitor='accuracy', patience=10),
    #                            monitor='val_accuracy', min_delta=0,#0.01, 
    #                                  patience=20,# verbose=2, 
    #                                  mode='max', restore_best_weights=True),#start_from_epoch=15),
    # #                                     tensorboard_callback,
    #                                     reduce_lr,
    #                         checkpoints
    #                        ],batch_size=batch_size)#50)

    
    return model

In [ ]:
def buildAndTrainModel(DLTechniqueName,x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels,batch_size,optimizer,datasetName=None,embeddingName=None,class_weights_train=None,class_weights_test=None,dataFormat=''):
    print("Started function execution of buildAndEvaluateModel")
    print("Batch size: ",batch_size)
#     if DLTechniqueName is 'CNNBILSTM':
#         model = KerasClassifier(build_fn=createModelCNNBILSTM,
# #                                 epochs=100, #batch_size=batch_size,#64,
#                                 verbose=True)#False)                                     
         
#     elif DLTechniqueName is 'CNNBIGRU':
#         model = KerasClassifier(build_fn=createModelCNNBIGRU,
# #                                 epochs=100,# batch_size=batch_size,#64,
#                                 verbose=True)
#     elif DLTechniqueName is 'CNNBILSTM2':
#         model = KerasClassifier(build_fn=createModelCNNBILSTM2,
# #                                 epochs=100,# batch_size=batch_size,#64,
#                                 verbose=True)                                     
#     elif DLTechniqueName is 'CNNBIGRU2':
#         model = KerasClassifier(build_fn=createModelCNNBIGRU2,
# #                                 epochs=100,# batch_size=batch_size,#64,
#                                 verbose=True)                                     
# #         model = KerasClassifier(build_fn=createModelCNNBIGRU2,
# #                                 epochs=1, batch_size=batch_size,#64,
# #                                 verbose=False)
# #         return createModelCNNBIGRU2(DLTechniqueName,x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels,batch_size,optimizer,datasetName,embeddingName,class_weights)
#     elif DLTechniqueName is 'CNNBILSTM3':
#         model = KerasClassifier(build_fn=createModelCNNBILSTM3,
# #                                 epochs=100,# batch_size=batch_size,#64,
#                                 verbose=True)
#     elif DLTechniqueName is 'CNNBIGRU3':
#         model = KerasClassifier(build_fn=createModelCNNBIGRU3,
# #                                 epochs=100,# batch_size=batch_size,#64,
#                                 verbose=True)
    if DLTechniqueName not in ['CNNBILSTM','CNNBILSTM2','CNNBILSTM3','CNNBIGRU','CNNBIGRU2','CNNBIGRU3']:
        raise MyValidationError("Invalid input hybrid DL model name.")        
        
         
#     param_grid = dict(num_filters=[100],#128],#[128],#[64,128,256],#,512],#[32, 64, 128, 256],
#                       kernel_size=[3],#,5],#[3,5],#, 5],
#                       text_tokenizer=[text_tokenizer],
#                       embedding_dimension=[embedding_dimension],
#                       maximum_len=[maximum_len],
#                       matrix_embeddings=[matrix_embeddings],
#                      num_labels=[num_labels],
#                      optimizer=[optimizer])
#     grid = GridSearchCV(estimator=model, param_grid=param_grid,
#                                 cv=2, verbose=1)#, n_iter=5)
#     import datetime
#     log_dir = "logs\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
#     tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir, histogram_freq=1)
#     26th Apr
#     reduce_lr = ReduceLROnPlateau(monitor='val_acc', mode='max',min_delta = 0.01, factor=0.1, patience=2, verbose=1, min_lr=0.0000000000000001)
#     reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', mode="max",factor=0.1, patience=5, verbose=1, min_lr=0.0000000000000001)#,min_delta=0.1)
#     model_name = 'model_'+DLTechniqueName+'_'+str(batch_size)+'_'+optimizer+'_'+datasetName+'_'+embeddingName+'_trained.h5'
#     checkpoints = ModelCheckpoint(model_name, monitor="val_accuracy", mode="max", verbose=True, save_best_only=True)
    # 26th Apri
    with tf.device("/TPU:0"):
        print("Started model training:")
        
        reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', mode="max",factor=0.1, patience=8, verbose=0, min_lr=0.0000000000000001)#,min_delta=0.01)
        num_filters = [100]#,256]
        num_kernels = [3]#,5],7]
#         model.compile( loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
        if DLTechniqueName not in ['CNNBILSTM2','CNNBIGRU2']:
            for num_filter in num_filters:
                for num_kernel in num_kernels:
                    model_name = 'model_dataFormat'+dataFormat+'_'+DLTechniqueName+'_batchsize_'+str(batch_size)+'_filters_'+str(num_filter)+'_kernels_'+str(num_kernel)+'_opt_'+optimizer+'_'+datasetName+'_'+embeddingName+'_trained.h5'
                    checkpoints = ModelCheckpoint(model_name, monitor="val_accuracy",  mode='max',verbose=False, save_best_only=True)
                    if DLTechniqueName is 'CNNBILSTM':
                        model = createModelCNNBILSTM(num_filter, num_kernel, text_tokenizer, embedding_dimension, maximum_len,matrix_embeddings,num_labels,optimizer)
                    elif DLTechniqueName is 'CNNBILSTM3':
                        model = createModelCNNBILSTM3(num_filter, num_kernel, text_tokenizer, embedding_dimension, maximum_len,matrix_embeddings,num_labels,optimizer)                            
                    elif DLTechniqueName is 'CNNBIGRU':
                        model = createModelCNNBIGRU(num_filter, num_kernel, text_tokenizer, embedding_dimension, maximum_len,matrix_embeddings,num_labels,optimizer)
                    elif DLTechniqueName is 'CNNBIGRU3':
                        model = createModelCNNBIGRU3(num_filter, num_kernel, text_tokenizer, embedding_dimension, maximum_len,matrix_embeddings,num_labels,optimizer)                    

                    model.fit(x_train, y_train, class_weight = class_weights_train,validation_data= (x_test,y_test),epochs=100,verbose=0,
                        callbacks=[tf.keras.callbacks.EarlyStopping(
        #                                 monitor='accuracy', patience=10),
                                   monitor='val_accuracy', min_delta=0,#0.01, 
                                         patience=20,# verbose=2, 
                                         mode='max', restore_best_weights=True),#start_from_epoch=15),
        #                                     tensorboard_callback,
                                            reduce_lr,
                                checkpoints
                               ],batch_size=batch_size)#50)
                    return model
        else:
            print("In the other part")
            
            for num_filter in num_filters:
                for num_kernel in num_kernels:
                    model_name = 'model_dataFormat'+dataFormat+'_'+DLTechniqueName+'_batchsize_'+str(batch_size)+'_filters_'+str(num_filter)+'_kernels_'+str(num_kernel)+'_opt_'+optimizer+'_'+datasetName+'_'+embeddingName+'_trained.h5'
                    checkpoints = ModelCheckpoint(model_name, monitor="val_accuracy",  mode='max',verbose=False, save_best_only=True)
                    
                    if DLTechniqueName is 'CNNBIGRU2':                        
                        model = createModelCNNBIGRU2(num_filter, num_kernel, text_tokenizer, embedding_dimension, maximum_len,matrix_embeddings,num_labels,optimizer)
                    elif DLTechniqueName is 'CNNBILSTM2':
                        model = createModelCNNBILSTM2(num_filter, num_kernel, text_tokenizer, embedding_dimension, maximum_len,matrix_embeddings,num_labels,optimizer)
                    model.fit([x_train,x_train], y_train, class_weight = class_weights_train,validation_data= ([x_test,x_test],y_test),epochs=100,verbose=0,
                    callbacks=[tf.keras.callbacks.EarlyStopping(
    #                                 monitor='accuracy', patience=10),
                               monitor='val_accuracy', min_delta=0,#0.01, 
                                     patience=20,# verbose=2, 
                                     mode='max', restore_best_weights=True),#start_from_epoch=15),
    #                                     tensorboard_callback,
                                        reduce_lr,
                            checkpoints
                           ],batch_size=batch_size)#50)
                    return model

#         if DLTechniqueName is 'CNNBIGRU2':
#             grid_result = grid.fit(x_train, y_train, validation_data= (x_test,y_test),
#                             callbacks=[tf.keras.callbacks.EarlyStopping(
# #                                 monitor='accuracy', patience=10),
#                                        monitor='val_accuracy', min_delta=0, 
#                                              patience=30, verbose=2, 
#                                              mode='max', restore_best_weights=True),
# #                                     tensorboard_callback,
#                                     reduce_lr,
#                                     checkpoints
#                                    ])
#         else:
#             grid_result = grid.fit(x_train, y_train, validation_data= (x_test,y_test),
#                                 callbacks=[tf.keras.callbacks.EarlyStopping(
# #                                     monitor='accuracy', 
# #                                     patience=10,
# #                                 min_delta = 0,
#                                 monitor='val_accuracy', min_delta=0,#0.01, 
#                                  patience=20, verbose=2, 
#                                  mode='max', restore_best_weights=True),#start_from_epoch=15),
# #                                         tensorboard_callback,
#                                         reduce_lr,
#                                         checkpoints],
#                                   batch_size=50
#                                   )
    return model
#     return

In [ ]:
def evaluateModel(grid_result,x_holdout,y_holdout,label_encoder,model=None):
    #         grid_result.best_estimator_.save('bestCNNBILSTM.h5')
    if model not in ['CNNBILSTM2','CNNBIGRU2']:
        y_pred = grid_result.predict(x_holdout)
    else:
        y_pred = grid_result.predict([x_holdout,x_holdout])
    print("Dimension of y_pred: ",y_pred.shape)
    print("Dimension of y_holdout: ",y_holdout.shape)
#     print("y_holdout values: ",y_holdout)
#     print("y_pred values:",y_pred)
#     print("y_pred values sample: ",y_pred.head())
    y_true = [label_encoder.classes_[x] for x in y_holdout]#.argmax(axis=1)]
#         y_predicted = [label_encoder.classes_[x] for x in y_pred.argmax(axis=1)]
    y_predicted = [label_encoder.classes_[x] for x in y_pred.argmax(axis=1)]

    print(classification_report(y_true,y_predicted))
#     if model is None:
#         print('Best number of filters: ',grid_result.best_params_['num_filters'])
#         print('Best kernel size: ',grid_result.best_params_['kernel_size'])
#         print('Best embedding dimension: ',grid_result.best_params_['embedding_dimension'])
#         print('Best training accuracy: ',grid_result.best_score_)
    print('Accuracy :',accuracy_score(y_true,y_predicted))
    print('F1 score :',f1_score(y_true, y_predicted, average='weighted'))
    print('Precision score is:',precision_score(y_true,y_predicted,average='weighted'))
    print('Recall score is:',recall_score(y_true,y_predicted,average='weighted'))
    
#     print()
#     print('Best parameters: ',grid_result.best_params_)
#             test_accuracy
    weighted_roc_auc_ovr = roc_auc_score( y_holdout,y_pred,multi_class="ovr",average="weighted",)
    print(f"Weighted AUROC score:{weighted_roc_auc_ovr:.2f}")
    return
    
    

In [ ]:
def getData(datasetName,embeddingName,approach): #,DLTechniqueName):
    print("Started function getDataAndTrainModel")
#     nltk.download('wordnet')
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension, label_encoder, num_labels, class_weights_train, class_weights_test = readDataAndPreprocess(datasetName,approach)
    
    if embeddingName is 'GLOVE':
        with tf.device("/TPU:0"):
            matrix_embeddings = getEmbeddingMatrixForGlove(text_tokenizer, embedding_dimension,indexed_embeddings_glove)
    elif embeddingName is 'WORD2VEC':
        #
        matrix_embeddings = getEmbeddingMatrixForWord2Vec(text_tokenizer, embedding_dimension,embeddings_word2vec)
    elif embeddingName is 'FASTTEXT':
        #
        matrix_embeddings = getEmbeddingMatrixForFasttext(text_tokenizer, embedding_dimension,embeddings_fasttext)
    else:
        raise MyValidationError("Invalid input word embedding technique name.")        
#     return buildAndEvaluateModel(DLTechniqueName,x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder)
    return x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, class_weights_train, class_weights_test

In [ ]:
# !mkdir nltk_data

In [ ]:
nltk.download("wordnet", "/root/nltk_data")

In [ ]:
nltk.download('wordnet')

In [ ]:
#  !unzip /root/nltk_data/corpora/wordnet.zip -d /root/nltk_data/corpora/

In [ ]:
# with tf.device("/TPU:0"):
#     x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, class_weights_train, class_weights_test = getData('MBTI','GLOVE','new')#,'CNNBIGRU')


In [ ]:
# class_weights_train

In [ ]:
# class_weights[0:][0:]

In [ ]:
# print({i for i in range(5)})

In [ ]:
# class_weights_new = {i : class_weights[i] for i in range(16)}

In [ ]:
# class_weights_new

In [ ]:
# y_pred = [label_encoder.classes_[x] for x in y_holdout]

In [ ]:
# with tf.device("/TPU:0"):
#     x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, class_weights_train, class_weights_test = getData('MBTI','WORD2VEC','old')


In [ ]:
# newModel = keras.models.load_model('/kaggle/working/model_dataFormatnew_CNNBILSTM_batchsize_50_filters_100_kernels_3_opt_Adam_MBTI_GLOVE_trained.h5')
# evaluateModel(model,x_holdout,y_holdout,label_encoder,model='CNNBILSTM')

In [ ]:
dataFormats = ['new']#,'old']
datasets = ['MBTI']
techniques = ['CNNBILSTM3','CNNBIGRU3']
batches = [50,100]
embeddings = ['WORD2VEC']#['GLOVE','FASTTEXT','WORD2VEC']
optimizers = ['adam']#,'SGD']

In [ ]:
# tf.Variable(1,dtype='int64')

In [181]:
with tf.device("/TPU:0"):
    for dataFormat in dataFormats:
        for dataset in datasets:
            for embedding in embeddings:
                for technique in techniques:
                    for num_batch in batches:
                        for optimizer in optimizers:
#                             print('Processing started for dataformat: '+dataFormat+' ,dataset: '+dataset+' ,embedding: '+embedding+' ,technique: '+technique+' ,batch_size: '+str(num_batch)+' ,optimizer: '+optimizer)
                            x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, class_weights_train, class_weights_test = getData(dataset,embedding,dataFormat)
                            model = buildAndTrainModel(technique,x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, num_batch, 'Adam',dataset,embedding,class_weights_train,class_weights_test,dataFormat)#,'new')
#                           modelName = dataFormat+'_'+dataset+'_'+embedding+'_'+technique+'_'+str(num_batch)+'_'+optimizer+'.h5'
                            print('Processing started for dataformat: '+dataFormat+' ,dataset: '+dataset+' ,embedding: '+embedding+' ,technique: '+technique+' ,batch_size: '+str(num_batch)+' ,optimizer: '+optimizer)
                            evaluateModel(model,x_holdout,y_holdout,label_encoder,model=technique)
#                             model.save('ISEARdummy.keras')

Started function getDataAndTrainModel
Started function exeuction of readDataAndPreprocess
Started execution of function preProcessDataMBTI
Number of columns: Index(['type', 'posts'], dtype='object')
Number of records: 8675
Number of records with personality type assigned:  8675
Class-wise break-up of personality types:
 type
INFP    1832
INFJ    1470
INTP    1304
INTJ    1091
ENTP     685
ENFP     675
ISTP     337
ISFP     271
ENTJ     231
ISTJ     205
ENFJ     190
ISFJ     166
ESTP      89
ESFP      48
ESFJ      42
ESTJ      39
Name: count, dtype: int64
Sample tokens 
 {'<OOV>': 1, 'like': 2, 'think': 3, 'people': 4, 'would': 5, 'know': 6, 'really': 7, 'thing': 8, 'time': 9, 'feel': 10, 'type': 11, 'make': 12, 'much': 13, 'well': 14, 'friend': 15}
Sample training data padded length is:  785
Shape of training data with padding:  (6072, 785)
Shape of testing data with padding:  (1744, 785)
Shape of holdout data with padding:  (859, 785)
Shape of training labels encoded:  (6072,)
Shape o

KeyboardInterrupt: 

In [182]:
print('test')

test


In [ ]:
dataFormats = ['new']#,'old']
datasets = ['TEC','ISEAR','MBTI']
techniques = ['CNNBILSTM','CNNBIGRU','CNNBILSTM2','CNNBIGRU2','CNNBILSTM3','CNNBIGRU3']
batches = [50,100]
embeddings = ['GLOVE','FASTTEXT','WORD2VEC']
optimizers = ['adam']#,'SGD']

In [ ]:
with tf.device("/TPU:0"):
    for dataFormat in dataFormats:
        for dataset in datasets:
            for embedding in embeddings:
                for technique in techniques:
                    for num_batch in batches:
                        for optimizer in optimizers:
                            print('Processing started for dataformat: '+dataFormat+' ,dataset: '+dataset+' ,embedding: '+embedding+' ,technique: '+technique+' ,batch_size: '+str(num_batch)+' ,optimizer: '+optimizer)
                            x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, class_weights_train, class_weights_test = getData(dataset,embedding,dataFormat)
                            model = buildAndTrainModel(technique,x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, num_batch, 'Adam',dataset,embedding,class_weights_train,class_weights_test,dataFormat)#,'new')
#                           modelName = dataFormat+'_'+dataset+'_'+embedding+'_'+technique+'_'+str(num_batch)+'_'+optimizer+'.h5'
                            evaluateModel(model,x_holdout,y_holdout,label_encoder,model=technique)
#                             model.save('ISEARdummy.keras')

In [ ]:
dataFormats = ['old']
datasets = ['TEC','ISEAR','MBTI']
techniques = ['CNNBILSTM','CNNBIGRU','CNNBILSTM2','CNNBIGRU2','CNNBILSTM3','CNNBIGRU3']
batches = [50,100]
embeddings = ['GLOVE','FASTTEXT','WORD2VEC']
optimizers = ['adam']#,'SGD']

In [ ]:
with tf.device("/TPU:0"):
    for dataFormat in dataFormats:
        for dataset in datasets:
            for embedding in embeddings:
                for technique in techniques:
                    for num_batch in batches:
                        for optimizer in optimizers:
                            print('Processing started for dataformat: '+dataFormat+' ,dataset: '+dataset+' ,embedding: '+embedding+' ,technique: '+technique+' ,batch_size: '+str(num_batch)+' ,optimizer: '+optimizer)
                            x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, class_weights_train, class_weights_test = getData(dataset,embedding,dataFormat)
                            model = buildAndTrainModel(technique,x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, num_batch, 'Adam',dataset,embedding,class_weights_train,class_weights_test,dataFormat)#,'new')
#                           modelName = dataFormat+'_'+dataset+'_'+embedding+'_'+technique+'_'+str(num_batch)+'_'+optimizer+'.h5'
                            evaluateModel(model,x_holdout,y_holdout,label_encoder,model=technique)
#                             model.save('ISEARdummy.keras')

In [ ]:
with tf.device("/TPU:0"):
#     for dataFormat in dataFormats:
#         for dataset in datasets:
#             for embedding in embeddings:
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, class_weights_train, class_weights_test = getData('ISEAR','GLOVE','new')
#                 for technique in techniques:
#                     for num_batch in batches:
#                         for optimizer in optimizers:
#                             print('Processing started for dataformat: '+dataFormat+' ,dataset: '+dataset+' ,embedding: '+embedding+' ,technique: '+technique+' ,batch_size: '+str(num_batch)+' ,optimizer: '+optimizer)
    model = buildAndTrainModel(technique,x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, num_batch, 'Adam',dataset,embedding,class_weights_train,class_weights_test,dataFormat)#,'new')
#     modelName = dataFormat+'_'+dataset+'_'+embedding+'_'+technique+'_'+str(num_batch)+'_'+optimizer+'.h5'
    evaluateModel(model,x_holdout,y_holdout,label_encoder,model=technique)
#     model.save('tecdummy.keras')

In [ ]:
with tf.device("/TPU:0"):
    for dataFormat in dataFormats:
        for dataset in datasets:
            for embedding in embeddings:
                x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, class_weights_train, class_weights_test = getData(dataset,embedding,dataFormat)
                for technique in techniques:
                    for num_batch in batches:
                        for optimizer in optimizers:
                            print('Processing started for dataformat: '+dataFormat+' ,dataset: '+dataset+' ,embedding: '+embedding+' ,technique: '+technique+' ,batch_size: '+str(num_batch)+' ,optimizer: '+optimizer)
                            model = buildAndTrainModel(technique,x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, num_batch, 'Adam',dataset,embedding,class_weights_train,class_weights_test,dataFormat)#,'new')
                            modelName = dataFormat+'_'+dataset+'_'+embedding+'_'+technique+'_'+str(num_batch)+'_'+optimizer+'.h5'
                            evaluateModel(model,x_holdout,y_holdout,label_encoder,model=technique)
                            model.save(modelName)
                            

In [ ]:
evaluateModel(grid_result_cnnbilstm_mbti_glove_adam_50,x_holdout,y_holdout,label_encoder,model='dd')

In [ ]:
with tf.device("/TPU:0"):
    grid_result_cnnbilstm_mbti_glove_adam_100 = buildAndTrainModel('CNNBILSTM',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 50, 'Adam','MBTI','GLOVE')

In [ ]:
print(type(matrix_embeddings.shape))
shape1 = list(matrix_embeddings.shape)
print(shape1)

In [ ]:
with tf.device("/TPU:0"):
    grid_result_cnnbigru2_isear_fasttext_adam = buildAndTrainModel('CNNBIGRU2',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 64, 'Adam','ISEAR','FASTTEXT')



In [ ]:
evaluateModel(grid_result_cnnbigru2_isear_fasttext_adam,x_holdout,y_holdout,label_encoder,model='dd')

In [ ]:
with tf.device("/TPU:0"):
    grid_result_cnnbigru2_isear_fasttext_adam = buildAndTrainModel('CNNBILSTM',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 64, 'Adam','ISEAR','FASTTEXT')


In [ ]:
evaluateModel(grid_result_cnnbilstm_isear_fasttext_adam,x_holdout,y_holdout,label_encoder,model=None)

In [ ]:
with tf.device("/TPU:0"):
    grid_result_cnnbigru_isear_fasttext_adam = buildAndTrainModel('CNNBIGRU',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 64, 'Adam','ISEAR','FASTTEXT')


In [ ]:
evaluateModel(grid_result_cnnbigru_isear_fasttext_adam,x_holdout,y_holdout,label_encoder,model=None)

In [ ]:
with tf.device("/TPU:0"):
    x_train_TEC, x_test_TEC, x_holdout_TEC, y_train_TEC, y_test_TEC, y_holdout_TEC, text_tokenizer_TEC, maximum_len_TEC, embedding_dimension_TEC,matrix_embeddings_TEC, label_encoder_TEC, num_labels_TEC = getData('TEC','FASTTEXT')#,'CNNBIGRU')

In [ ]:
with tf.device("/TPU:0"):
    grid_result_cnnbigru2_tec_fasttext_64_adam = buildAndTrainModel('CNNBIGRU2',x_train_TEC, x_test_TEC, x_holdout_TEC, y_train_TEC, y_test_TEC, y_holdout_TEC, text_tokenizer_TEC, maximum_len_TEC, embedding_dimension_TEC,matrix_embeddings_TEC, label_encoder_TEC, num_labels_TEC, 64, 'Adam','TEC','FASTTEXT')



In [ ]:
with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('ISEAR','FASTTEXT')#,'CNNBIGRU')
    grid_result_cnnbilstm_isear_fasttext_32_adam = buildAndTrainModel('CNNBILSTM',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 32, 'Adam')
evaluateModel(grid_result_cnnbilstm_isear_fasttext_32_adam,x_holdout,y_holdout,label_encoder)


In [ ]:


with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('ISEAR','FASTTEXT')#,'CNNBIGRU')
    grid_result_cnnbilstm_isear_fasttext_32_sgd = buildAndTrainModel('CNNBILSTM',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 32, 'SGD')
evaluateModel(grid_result_cnnbilstm_isear_fasttext_32_sgd,x_holdout,y_holdout,label_encoder)



In [ ]:

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('ISEAR','FASTTEXT')#,'CNNBIGRU')
    grid_result_cnnbilstm_isear_fasttext_64_adam = buildAndTrainModel('CNNBILSTM',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 64, 'Adam')
evaluateModel(grid_result_cnnbilstm_isear_fasttext_64_adam,x_holdout,y_holdout,label_encoder)


In [ ]:

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('ISEAR','FASTTEXT')#,'CNNBIGRU')
    grid_result_cnnbilstm_isear_fasttext_64_sgd = buildAndTrainModel('CNNBILSTM',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 64, 'SGD')
evaluateModel(grid_result_cnnbilstm_isear_fasttext_64_sgd,x_holdout,y_holdout,label_encoder)


In [ ]:
with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('ISEAR','FASTTEXT')#,'CNNBIGRU')
    grid_result_cnnbigru_isear_fasttext_32_adam = buildAndTrainModel('CNNBIGRU',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 32, 'Adam')
evaluateModel(grid_result_cnnbigru_isear_fasttext_32_adam,x_holdout,y_holdout,label_encoder)


In [ ]:

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('ISEAR','FASTTEXT')#,'CNNBIGRU')
    grid_result_cnnbigru_isear_fasttext_32_sgd = buildAndTrainModel('CNNBIGRU',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 32, 'SGD')
evaluateModel(grid_result_cnnbigru_isear_fasttext_32_sgd,x_holdout,y_holdout,label_encoder)


In [ ]:

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('ISEAR','FASTTEXT')#,'CNNBIGRU')
    grid_result_cnnbigru_isear_fasttext_64_adam = buildAndTrainModel('CNNBIGRU',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 64, 'Adam')
evaluateModel(grid_result_cnnbigru_isear_fasttext_64_adam,x_holdout,y_holdout,label_encoder)


In [ ]:

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('ISEAR','FASTTEXT')#,'CNNBIGRU')
    grid_result_cnnbigru_isear_fasttext_64_sgd = buildAndTrainModel('CNNBIGRU',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 64, 'SGD')
evaluateModel(grid_result_cnnbigru_isear_fasttext_64_sgd,x_holdout,y_holdout,label_encoder)


In [ ]:
with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('ISEAR','WORD2VEC')#,'CNNBIGRU')
    grid_result_cnnbilstm_isear_word2vec_32_adam = buildAndTrainModel('CNNBILSTM',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 32, 'Adam')
evaluateModel(grid_result_cnnbilstm_isear_word2vec_32_adam,x_holdout,y_holdout,label_encoder)


In [ ]:

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('ISEAR','WORD2VEC')#,'CNNBIGRU')
    grid_result_cnnbilstm_isear_word2vec_32_sgd = buildAndTrainModel('CNNBILSTM',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 32, 'SGD')
evaluateModel(grid_result_cnnbilstm_isear_word2vec_32_sgd,x_holdout,y_holdout,label_encoder)


In [ ]:

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('ISEAR','WORD2VEC')#,'CNNBIGRU')
    grid_result_cnnbilstm_isear_word2vec_64_adam = buildAndTrainModel('CNNBILSTM',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 64, 'Adam')
evaluateModel(grid_result_cnnbilstm_isear_word2vec_64_adam,x_holdout,y_holdout,label_encoder)


In [ ]:

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('ISEAR','WORD2VEC')#,'CNNBIGRU')
    grid_result_cnnbilstm_isear_word2vec_64_sgd = buildAndTrainModel('CNNBILSTM',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 64, 'SGD')
evaluateModel(grid_result_cnnbilstm_isear_word2vec_64_sgd,x_holdout,y_holdout,label_encoder)


In [ ]:
with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('ISEAR','WORD2VEC')#,'CNNBIGRU')
    grid_result_cnnbigru_isear_word2vec_32_adam = buildAndTrainModel('CNNBIGRU',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 32, 'Adam')
evaluateModel(grid_result_cnnbigru_isear_word2vec_32_adam,x_holdout,y_holdout,label_encoder)


In [ ]:

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('ISEAR','WORD2VEC')#,'CNNBIGRU')
    grid_result_cnnbigru_isear_word2vec_32_sgd = buildAndTrainModel('CNNBIGRU',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 32, 'SGD')
evaluateModel(grid_result_cnnbigru_isear_word2vec_32_sgd,x_holdout,y_holdout,label_encoder)


In [ ]:

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('ISEAR','WORD2VEC')#,'CNNBIGRU')
    grid_result_cnnbigru_isear_word2vec_64_adam = buildAndTrainModel('CNNBIGRU',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 64, 'Adam')
evaluateModel(grid_result_cnnbigru_isear_word2vec_64_adam,x_holdout,y_holdout,label_encoder)


In [ ]:

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('ISEAR','WORD2VEC')#,'CNNBIGRU')
    grid_result_cnnbigru_isear_word2vec_64_sgd = buildAndTrainModel('CNNBIGRU',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 64, 'SGD')
evaluateModel(grid_result_cnnbigru_isear_word2vec_64_sgd,x_holdout,y_holdout,label_encoder)


In [ ]:
with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('ISEAR','GLOVE')#,'CNNBIGRU')
    grid_result_cnnbilstm_isear_glove_32_adam = buildAndTrainModel('CNNBILSTM',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 32, 'Adam')
evaluateModel(grid_result_cnnbilstm_isear_glove_32_adam,x_holdout,y_holdout,label_encoder)


In [ ]:

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('ISEAR','GLOVE')#,'CNNBIGRU')
    grid_result_cnnbilstm_isear_glove_32_sgd = buildAndTrainModel('CNNBILSTM',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 32, 'SGD')
evaluateModel(grid_result_cnnbilstm_isear_glove_32_sgd,x_holdout,y_holdout,label_encoder)


In [ ]:

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('ISEAR','GLOVE')#,'CNNBIGRU')
    grid_result_cnnbilstm_isear_glove_64_adam = buildAndTrainModel('CNNBILSTM',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 64, 'Adam')
evaluateModel(grid_result_cnnbilstm_isear_glove_64_adam,x_holdout,y_holdout,label_encoder)


In [ ]:

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('ISEAR','GLOVE')#,'CNNBIGRU')
    grid_result_cnnbilstm_isear_glove_64_sgd = buildAndTrainModel('CNNBILSTM',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 64, 'SGD')
evaluateModel(grid_result_cnnbilstm_isear_glove_64_sgd,x_holdout,y_holdout,label_encoder)


In [ ]:
with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('ISEAR','GLOVE')#,'CNNBIGRU')
    grid_result_cnnbigru_isear_glove_32_adam = buildAndTrainModel('CNNBIGRU',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 32, 'Adam')
evaluateModel(grid_result_cnnbigru_isear_glove_32_adam,x_holdout,y_holdout,label_encoder)


In [ ]:

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('ISEAR','GLOVE')#,'CNNBIGRU')
    grid_result_cnnbigru_isear_glove_32_sgd = buildAndTrainModel('CNNBIGRU',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 32, 'SGD')
evaluateModel(grid_result_cnnbigru_isear_glove_32_sgd,x_holdout,y_holdout,label_encoder)


In [ ]:

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('ISEAR','GLOVE')#,'CNNBIGRU')
    grid_result_cnnbigru_isear_glove_64_adam = buildAndTrainModel('CNNBIGRU',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 64, 'Adam')
evaluateModel(grid_result_cnnbigru_isear_glove_64_adam,x_holdout,y_holdout,label_encoder)


In [ ]:

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('ISEAR','GLOVE')#,'CNNBIGRU')
    grid_result_cnnbigru_isear_glove_64_sgd = buildAndTrainModel('CNNBIGRU',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 64, 'SGD')
evaluateModel(grid_result_cnnbigru_isear_glove_64_sgd,x_holdout,y_holdout,label_encoder)

In [ ]:
with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('TEC','FASTTEXT')#,'CNNBIGRU')
    grid_result_cnnbilstm_tec_fasttext_32_adam = buildAndTrainModel('CNNBILSTM',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 32, 'Adam')
evaluateModel(grid_result_cnnbilstm_tec_fasttext_32_adam,x_holdout,y_holdout,label_encoder)

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('TEC','FASTTEXT')#,'CNNBIGRU')
    grid_result_cnnbilstm_tec_fasttext_32_sgd = buildAndTrainModel('CNNBILSTM',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 32, 'SGD')
evaluateModel(grid_result_cnnbilstm_tec_fasttext_32_sgd,x_holdout,y_holdout,label_encoder)

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('TEC','FASTTEXT')#,'CNNBIGRU')
    grid_result_cnnbilstm_tec_fasttext_64_adam = buildAndTrainModel('CNNBILSTM',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 64, 'Adam')
evaluateModel(grid_result_cnnbilstm_tec_fasttext_64_adam,x_holdout,y_holdout,label_encoder)

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('TEC','FASTTEXT')#,'CNNBIGRU')
    grid_result_cnnbilstm_tec_fasttext_64_sgd = buildAndTrainModel('CNNBILSTM',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 64, 'SGD')
evaluateModel(grid_result_cnnbilstm_tec_fasttext_64_sgd,x_holdout,y_holdout,label_encoder)
with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('TEC','FASTTEXT')#,'CNNBIGRU')
    grid_result_cnnbigru_tec_fasttext_32_adam = buildAndTrainModel('CNNBIGRU',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 32, 'Adam')
evaluateModel(grid_result_cnnbigru_tec_fasttext_32_adam,x_holdout,y_holdout,label_encoder)

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('TEC','FASTTEXT')#,'CNNBIGRU')
    grid_result_cnnbigru_tec_fasttext_32_sgd = buildAndTrainModel('CNNBIGRU',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 32, 'SGD')
evaluateModel(grid_result_cnnbigru_tec_fasttext_32_sgd,x_holdout,y_holdout,label_encoder)

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('TEC','FASTTEXT')#,'CNNBIGRU')
    grid_result_cnnbigru_tec_fasttext_64_adam = buildAndTrainModel('CNNBIGRU',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 64, 'Adam')
evaluateModel(grid_result_cnnbigru_tec_fasttext_64_adam,x_holdout,y_holdout,label_encoder)

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('TEC','FASTTEXT')#,'CNNBIGRU')
    grid_result_cnnbigru_tec_fasttext_64_sgd = buildAndTrainModel('CNNBIGRU',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 64, 'SGD')
evaluateModel(grid_result_cnnbigru_tec_fasttext_64_sgd,x_holdout,y_holdout,label_encoder)
with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('TEC','WORD2VEC')#,'CNNBIGRU')
    grid_result_cnnbilstm_tec_word2vec_32_adam = buildAndTrainModel('CNNBILSTM',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 32, 'Adam')
evaluateModel(grid_result_cnnbilstm_tec_word2vec_32_adam,x_holdout,y_holdout,label_encoder)

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('TEC','WORD2VEC')#,'CNNBIGRU')
    grid_result_cnnbilstm_tec_word2vec_32_sgd = buildAndTrainModel('CNNBILSTM',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 32, 'SGD')
evaluateModel(grid_result_cnnbilstm_tec_word2vec_32_sgd,x_holdout,y_holdout,label_encoder)

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('TEC','WORD2VEC')#,'CNNBIGRU')
    grid_result_cnnbilstm_tec_word2vec_64_adam = buildAndTrainModel('CNNBILSTM',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 64, 'Adam')
evaluateModel(grid_result_cnnbilstm_tec_word2vec_64_adam,x_holdout,y_holdout,label_encoder)

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('TEC','WORD2VEC')#,'CNNBIGRU')
    grid_result_cnnbilstm_tec_word2vec_64_sgd = buildAndTrainModel('CNNBILSTM',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 64, 'SGD')
evaluateModel(grid_result_cnnbilstm_tec_word2vec_64_sgd,x_holdout,y_holdout,label_encoder)
with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('TEC','WORD2VEC')#,'CNNBIGRU')
    grid_result_cnnbigru_tec_word2vec_32_adam = buildAndTrainModel('CNNBIGRU',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 32, 'Adam')
evaluateModel(grid_result_cnnbigru_tec_word2vec_32_adam,x_holdout,y_holdout,label_encoder)

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('TEC','WORD2VEC')#,'CNNBIGRU')
    grid_result_cnnbigru_tec_word2vec_32_sgd = buildAndTrainModel('CNNBIGRU',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 32, 'SGD')
evaluateModel(grid_result_cnnbigru_tec_word2vec_32_sgd,x_holdout,y_holdout,label_encoder)

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('TEC','WORD2VEC')#,'CNNBIGRU')
    grid_result_cnnbigru_tec_word2vec_64_adam = buildAndTrainModel('CNNBIGRU',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 64, 'Adam')
evaluateModel(grid_result_cnnbigru_tec_word2vec_64_adam,x_holdout,y_holdout,label_encoder)

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('TEC','WORD2VEC')#,'CNNBIGRU')
    grid_result_cnnbigru_tec_word2vec_64_sgd = buildAndTrainModel('CNNBIGRU',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 64, 'SGD')
evaluateModel(grid_result_cnnbigru_tec_word2vec_64_sgd,x_holdout,y_holdout,label_encoder)
with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('TEC','GLOVE')#,'CNNBIGRU')
    grid_result_cnnbilstm_tec_glove_32_adam = buildAndTrainModel('CNNBILSTM',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 32, 'Adam')
evaluateModel(grid_result_cnnbilstm_tec_glove_32_adam,x_holdout,y_holdout,label_encoder)

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('TEC','GLOVE')#,'CNNBIGRU')
    grid_result_cnnbilstm_tec_glove_32_sgd = buildAndTrainModel('CNNBILSTM',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 32, 'SGD')
evaluateModel(grid_result_cnnbilstm_tec_glove_32_sgd,x_holdout,y_holdout,label_encoder)

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('TEC','GLOVE')#,'CNNBIGRU')
    grid_result_cnnbilstm_tec_glove_64_adam = buildAndTrainModel('CNNBILSTM',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 64, 'Adam')
evaluateModel(grid_result_cnnbilstm_tec_glove_64_adam,x_holdout,y_holdout,label_encoder)

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('TEC','GLOVE')#,'CNNBIGRU')
    grid_result_cnnbilstm_tec_glove_64_sgd = buildAndTrainModel('CNNBILSTM',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 64, 'SGD')
evaluateModel(grid_result_cnnbilstm_tec_glove_64_sgd,x_holdout,y_holdout,label_encoder)
with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('TEC','GLOVE')#,'CNNBIGRU')
    grid_result_cnnbigru_tec_glove_32_adam = buildAndTrainModel('CNNBIGRU',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 32, 'Adam')
evaluateModel(grid_result_cnnbigru_tec_glove_32_adam,x_holdout,y_holdout,label_encoder)

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('TEC','GLOVE')#,'CNNBIGRU')
    grid_result_cnnbigru_tec_glove_32_sgd = buildAndTrainModel('CNNBIGRU',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 32, 'SGD')
evaluateModel(grid_result_cnnbigru_tec_glove_32_sgd,x_holdout,y_holdout,label_encoder)

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('TEC','GLOVE')#,'CNNBIGRU')
    grid_result_cnnbigru_tec_glove_64_adam = buildAndTrainModel('CNNBIGRU',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 64, 'Adam')
evaluateModel(grid_result_cnnbigru_tec_glove_64_adam,x_holdout,y_holdout,label_encoder)

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('TEC','GLOVE')#,'CNNBIGRU')
    grid_result_cnnbigru_tec_glove_64_sgd = buildAndTrainModel('CNNBIGRU',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 64, 'SGD')
evaluateModel(grid_result_cnnbigru_tec_glove_64_sgd,x_holdout,y_holdout,label_encoder)



In [ ]:
with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('TEC','FASTTEXT')#,'CNNBIGRU')
    grid_result_cnnbigru_tec_fasttext_64_adam = buildAndTrainModel('CNNBIGRU',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 64, 'Adam')
evaluateModel(grid_result_cnnbigru_tec_fasttext_64_adam,x_holdout,y_holdout,label_encoder)

In [ ]:
with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('MBTI','FASTTEXT')#,'CNNBIGRU')
    grid_result_cnnbilstm_MBTI_fasttext_32_adam = buildAndTrainModel('CNNBILSTM',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 32, 'Adam')
evaluateModel(grid_result_cnnbilstm_MBTI_fasttext_32_adam,x_holdout,y_holdout,label_encoder)

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('MBTI','FASTTEXT')#,'CNNBIGRU')
    grid_result_cnnbilstm_MBTI_fasttext_32_sgd = buildAndTrainModel('CNNBILSTM',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 32, 'SGD')
evaluateModel(grid_result_cnnbilstm_MBTI_fasttext_32_sgd,x_holdout,y_holdout,label_encoder)

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('MBTI','FASTTEXT')#,'CNNBIGRU')
    grid_result_cnnbilstm_MBTI_fasttext_64_adam = buildAndTrainModel('CNNBILSTM',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 64, 'Adam')
evaluateModel(grid_result_cnnbilstm_MBTI_fasttext_64_adam,x_holdout,y_holdout,label_encoder)

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('MBTI','FASTTEXT')#,'CNNBIGRU')
    grid_result_cnnbilstm_MBTI_fasttext_64_sgd = buildAndTrainModel('CNNBILSTM',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 64, 'SGD')
evaluateModel(grid_result_cnnbilstm_MBTI_fasttext_64_sgd,x_holdout,y_holdout,label_encoder)
with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('MBTI','FASTTEXT')#,'CNNBIGRU')
    grid_result_cnnbigru_MBTI_fasttext_32_adam = buildAndTrainModel('CNNBIGRU',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 32, 'Adam')
evaluateModel(grid_result_cnnbigru_MBTI_fasttext_32_adam,x_holdout,y_holdout,label_encoder)

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('MBTI','FASTTEXT')#,'CNNBIGRU')
    grid_result_cnnbigru_MBTI_fasttext_32_sgd = buildAndTrainModel('CNNBIGRU',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 32, 'SGD')
evaluateModel(grid_result_cnnbigru_MBTI_fasttext_32_sgd,x_holdout,y_holdout,label_encoder)

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('MBTI','FASTTEXT')#,'CNNBIGRU')
    grid_result_cnnbigru_MBTI_fasttext_64_adam = buildAndTrainModel('CNNBIGRU',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 64, 'Adam')
evaluateModel(grid_result_cnnbigru_MBTI_fasttext_64_adam,x_holdout,y_holdout,label_encoder)

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('MBTI','FASTTEXT')#,'CNNBIGRU')
    grid_result_cnnbigru_MBTI_fasttext_64_sgd = buildAndTrainModel('CNNBIGRU',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 64, 'SGD')
evaluateModel(grid_result_cnnbigru_MBTI_fasttext_64_sgd,x_holdout,y_holdout,label_encoder)
with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('MBTI','WORD2VEC')#,'CNNBIGRU')
    grid_result_cnnbilstm_MBTI_word2vec_32_adam = buildAndTrainModel('CNNBILSTM',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 32, 'Adam')
evaluateModel(grid_result_cnnbilstm_MBTI_word2vec_32_adam,x_holdout,y_holdout,label_encoder)

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('MBTI','WORD2VEC')#,'CNNBIGRU')
    grid_result_cnnbilstm_MBTI_word2vec_32_sgd = buildAndTrainModel('CNNBILSTM',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 32, 'SGD')
evaluateModel(grid_result_cnnbilstm_MBTI_word2vec_32_sgd,x_holdout,y_holdout,label_encoder)

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('MBTI','WORD2VEC')#,'CNNBIGRU')
    grid_result_cnnbilstm_MBTI_word2vec_64_adam = buildAndTrainModel('CNNBILSTM',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 64, 'Adam')
evaluateModel(grid_result_cnnbilstm_MBTI_word2vec_64_adam,x_holdout,y_holdout,label_encoder)

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('MBTI','WORD2VEC')#,'CNNBIGRU')
    grid_result_cnnbilstm_MBTI_word2vec_64_sgd = buildAndTrainModel('CNNBILSTM',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 64, 'SGD')
evaluateModel(grid_result_cnnbilstm_MBTI_word2vec_64_sgd,x_holdout,y_holdout,label_encoder)
with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('MBTI','WORD2VEC')#,'CNNBIGRU')
    grid_result_cnnbigru_MBTI_word2vec_32_adam = buildAndTrainModel('CNNBIGRU',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 32, 'Adam')
evaluateModel(grid_result_cnnbigru_MBTI_word2vec_32_adam,x_holdout,y_holdout,label_encoder)

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('MBTI','WORD2VEC')#,'CNNBIGRU')
    grid_result_cnnbigru_MBTI_word2vec_32_sgd = buildAndTrainModel('CNNBIGRU',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 32, 'SGD')
evaluateModel(grid_result_cnnbigru_MBTI_word2vec_32_sgd,x_holdout,y_holdout,label_encoder)

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('MBTI','WORD2VEC')#,'CNNBIGRU')
    grid_result_cnnbigru_MBTI_word2vec_64_adam = buildAndTrainModel('CNNBIGRU',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 64, 'Adam')
evaluateModel(grid_result_cnnbigru_MBTI_word2vec_64_adam,x_holdout,y_holdout,label_encoder)

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('MBTI','WORD2VEC')#,'CNNBIGRU')
    grid_result_cnnbigru_MBTI_word2vec_64_sgd = buildAndTrainModel('CNNBIGRU',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 64, 'SGD')
evaluateModel(grid_result_cnnbigru_MBTI_word2vec_64_sgd,x_holdout,y_holdout,label_encoder)
with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('MBTI','GLOVE')#,'CNNBIGRU')
    grid_result_cnnbilstm_MBTI_glove_32_adam = buildAndTrainModel('CNNBILSTM',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 32, 'Adam')
evaluateModel(grid_result_cnnbilstm_MBTI_glove_32_adam,x_holdout,y_holdout,label_encoder)

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('MBTI','GLOVE')#,'CNNBIGRU')
    grid_result_cnnbilstm_MBTI_glove_32_sgd = buildAndTrainModel('CNNBILSTM',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 32, 'SGD')
evaluateModel(grid_result_cnnbilstm_MBTI_glove_32_sgd,x_holdout,y_holdout,label_encoder)

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('MBTI','GLOVE')#,'CNNBIGRU')
    grid_result_cnnbilstm_MBTI_glove_64_adam = buildAndTrainModel('CNNBILSTM',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 64, 'Adam')
evaluateModel(grid_result_cnnbilstm_MBTI_glove_64_adam,x_holdout,y_holdout,label_encoder)

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('MBTI','GLOVE')#,'CNNBIGRU')
    grid_result_cnnbilstm_MBTI_glove_64_sgd = buildAndTrainModel('CNNBILSTM',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 64, 'SGD')
evaluateModel(grid_result_cnnbilstm_MBTI_glove_64_sgd,x_holdout,y_holdout,label_encoder)
with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('MBTI','GLOVE')#,'CNNBIGRU')
    grid_result_cnnbigru_MBTI_glove_32_adam = buildAndTrainModel('CNNBIGRU',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 32, 'Adam')
evaluateModel(grid_result_cnnbigru_MBTI_glove_32_adam,x_holdout,y_holdout,label_encoder)

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('MBTI','GLOVE')#,'CNNBIGRU')
    grid_result_cnnbigru_MBTI_glove_32_sgd = buildAndTrainModel('CNNBIGRU',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 32, 'SGD')
evaluateModel(grid_result_cnnbigru_MBTI_glove_32_sgd,x_holdout,y_holdout,label_encoder)

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('MBTI','GLOVE')#,'CNNBIGRU')
    grid_result_cnnbigru_MBTI_glove_64_adam = buildAndTrainModel('CNNBIGRU',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 64, 'Adam')
evaluateModel(grid_result_cnnbigru_MBTI_glove_64_adam,x_holdout,y_holdout,label_encoder)

with tf.device("/TPU:0"):
    x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels = getData('MBTI','GLOVE')#,'CNNBIGRU')
    grid_result_cnnbigru_MBTI_glove_64_sgd = buildAndTrainModel('CNNBIGRU',x_train, x_test, x_holdout, y_train, y_test, y_holdout, text_tokenizer, maximum_len, embedding_dimension,matrix_embeddings, label_encoder, num_labels, 64, 'SGD')
evaluateModel(grid_result_cnnbigru_MBTI_glove_64_sgd,x_holdout,y_holdout,label_encoder)

